# Import

https://dacon.io/en/codeshare/2876


[MLP + NGB + XGB + CAT with Data Over Sampling (0.144)](https://dacon.io/codeshare/4711)

[model](https://github.com/cge1111/lg_aimers_antenna/blob/main/model/jongsoo.ipynb)

[catboost](https://dacon.io/competitions/official/235927/codeshare/6045?page=1&dtype=recent)

[RadomForest + optuna](https://dacon.io/en/competitions/official/235848/codeshare/4058)

**parms참고 -> [multioutput_github](https://github.com/scikit-learn/scikit-learn/blob/36958fb24/sklearn/multioutput.py#L153)

rf param -> [rf_githib](https://github.com/scikit-learn/scikit-learn/blob/36958fb24/sklearn/ensemble/_forest.py#L301)

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# pip install pycaret

In [21]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import sklearn.metrics as metrics 

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
import lightgbm as lgb  
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

In [22]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

# Data Load

%pwd

In [3]:
# # 'macbook' 
# base_path = '/Users/goeun/Documents/python/LGaimers/open/'

# train_df = pd.read_csv(base_path+'train.csv')
# test_df = pd.read_csv(base_path+'test.csv').iloc[:, 1:]
# submission = pd.read_csv(base_path+'sample_submission.csv')

In [3]:
base_path = '/Users/cge/Documents/python/LGaimers/open/'

train_df = pd.read_csv(base_path+'train.csv')
test_df = pd.read_csv(base_path+'test.csv')
submission = pd.read_csv(base_path+'sample_submission.csv')

# Model

XGB, LGBM, Random Forest Regressor

In [7]:
X = train_df.filter(regex='X') # Input : X Featrue
y = train_df.filter(regex='Y') # Output : Y Feature

remove_cols = ['X_02', 'X_04', 'X_23', 'X_46', 'X_47', 'X_48', 'X_10', 'X_11']

X = X.drop(columns=remove_cols)
test = test_df.drop(columns=remove_cols)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, shuffle=True)

In [8]:
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

(35646, 48) (3961, 48) (35646, 14) (3961, 14)


## Optuna

In [11]:
from tkinter.tix import Meter
import optuna
from optuna.samplers import TPESampler

sampler = TPESampler(seed=10)

def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt.iloc[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt.iloc[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score

def RF_objective(trial):
    max_depth = trial.suggest_int('max_depth', 3, 15)
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 1000)
    n_estimators =  trial.suggest_int('n_estimators', 100, 500)
    min_samples_split = trial.suggest_int('min_samples_split', 4, 10)
   
    model = MultiOutputRegressor(RandomForestRegressor(max_depth = max_depth, max_leaf_nodes = max_leaf_nodes,n_estimators = n_estimators,n_jobs=2,random_state=42))
    model.fit(X_train, y_train)    
    rmse = lg_nrmse(y_valid, model.predict(X_valid))
    return np.sqrt(rmse)
    
#Execute optuna and set hyperparameters
RF_study = optuna.create_study(direction='minimize', sampler=sampler, study_name='RandomForestRegressor')
RF_study.optimize(RF_objective, n_trials=200)

#Create an instance with tuned hyperparameters
# optimized_RF = MultiOutputRegressor(RandomForestClassifier(max_depth = RF_study.best_params['max_depth'], max_leaf_nodes = RF_study.best_params['max_leaf_nodes'],
#                                       n_estimators = RF_study.best_params['n_estimators'],n_jobs=2,random_state=25))

[I 2022-08-24 14:11:31,541] A new study created in memory with name: RandomForestRegressor
[I 2022-08-24 14:20:41,470] Trial 0 finished with value: 1.397303748087589 and parameters: {'max_depth': 13, 'max_leaf_nodes': 22, 'n_estimators': 354, 'min_samples_split': 9}. Best is trial 0 with value: 1.397303748087589.
[I 2022-08-24 14:27:49,774] Trial 1 finished with value: 1.3929313287625176 and parameters: {'max_depth': 9, 'max_leaf_nodes': 226, 'n_estimators': 179, 'min_samples_split': 9}. Best is trial 1 with value: 1.3929313287625176.
[I 2022-08-24 14:36:35,326] Trial 2 finished with value: 1.3964048644952114 and parameters: {'max_depth': 5, 'max_leaf_nodes': 90, 'n_estimators': 374, 'min_samples_split': 10}. Best is trial 1 with value: 1.3929313287625176.
[I 2022-08-24 14:42:40,446] Trial 3 finished with value: 1.400857803528424 and parameters: {'max_depth': 3, 'max_leaf_nodes': 513, 'n_estimators': 425, 'min_samples_split': 8}. Best is trial 1 with value: 1.3929313287625176.
[I 2022-

In [12]:
print('Best parameters: {}'.format(RF_study.best_params))
print('Best RMSE: {}'.format(RF_study.best_value))

Best parameters: {'max_depth': 15, 'max_leaf_nodes': 997, 'n_estimators': 260, 'min_samples_split': 4}
Best RMSE: 1.3904049233095859


In [14]:
test.drop(columns=['ID'],inplace=True)

In [16]:
from sklearn.model_selection import KFold

fold = KFold(5, shuffle=True, random_state=42)

oof_val_preds = np.zeros((X.shape[0], 14))
oof_test_preds = np.zeros((test.shape[0], 14))
# best_trial = RF_study.best_trial
# params = best_trial.params
# print(params)

def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt.iloc[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt.iloc[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score

for idx, (train_idx, valid_idx) in enumerate(fold.split(X, y)):
    print('#'*40, f'fold {idx + 1} / fold {fold.n_splits}', '#'*40)
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]
    X_valid, y_valid = X.loc[valid_idx], y.loc[valid_idx]
 
    rf_model = MultiOutputRegressor(RandomForestRegressor(max_depth = 15, max_leaf_nodes = 997, n_estimators = 260, min_samples_split = 4, random_state=42))
    model = rf_model.fit(X_train, y_train)
    oof_val_preds[valid_idx] += model.predict(X_valid)
    oof_test_preds += model.predict(test)/fold.n_splits
    
    score = lg_nrmse(y_valid, oof_val_preds[valid_idx])
    print(f'fold {idx+1} nrmse : {score}\n')

######################################## fold 1 / fold 5 ########################################
fold 1 nrmse : 1.9403140739809648

######################################## fold 2 / fold 5 ########################################
fold 2 nrmse : 1.975730608859738

######################################## fold 3 / fold 5 ########################################
fold 3 nrmse : 1.9530304784516783

######################################## fold 4 / fold 5 ########################################
fold 4 nrmse : 1.9530507116985456

######################################## fold 5 / fold 5 ########################################
fold 5 nrmse : 1.9599600228059808



In [17]:
lg_nrmse(y, oof_val_preds)

1.9566692681491908

In [18]:
pred = model.predict(test)

In [19]:
for idx, col in enumerate(submission.columns):
    if col=='ID':
        continue
    submission[col] = pred[:,idx-1]
print('Done.')

Done.


In [20]:
submission.to_csv('/Users/cge/Documents/python/LGaimers/submission/rf_optuna.csv', index=False)

**1. baseline_lgb** [public score] 1.9453620126